In [19]:
import pandas as pd
import numpy as np

In [26]:
data = np.loadtxt("dataCircle.txt", dtype=np.float64)
pd.read_csv("dataCircle.txt", names=['x', 'y', 'class'], delim_whitespace=True, dtype={"x": np.float64, "y": np.float64, "class": "category"})

,x,y,class
0,0.00000,0.00000,1.00000
1,-3.70962,-5.35210,1.00000
2,-1.42211,-2.40219,1.00000
3,-2.66972,4.30397,1.00000
4,1.55548,4.93590,1.00000
5,-1.59524,6.23187,1.00000
6,-5.55343,-1.67754,1.00000
7,-5.57887,-0.37315,1.00000
8,2.91409,4.75527,1.00000
9,-1.92258,-5.99233,1.00000


In [16]:
data

array([[ 0.     ,  0.     ,  1.     ],
       [-3.70962, -5.3521 ,  1.     ],
       [-1.42211, -2.40219,  1.     ],
       [-2.66972,  4.30397,  1.     ],
       [ 1.55548,  4.9359 ,  1.     ],
       [-1.59524,  6.23187,  1.     ],
       [-5.55343, -1.67754,  1.     ],
       [-5.57887, -0.37315,  1.     ],
       [ 2.91409,  4.75527,  1.     ],
       [-1.92258, -5.99233,  1.     ],
       [ 4.57485,  4.59031,  1.     ],
       [ 5.32024, -0.0782 ,  1.     ],
       [ 1.98235, -1.70504,  1.     ],
       [-1.67116,  2.00863,  1.     ],
       [ 2.39138,  0.11597,  1.     ],
       [-0.03928, -2.43152,  1.     ],
       [ 1.29275, -1.86318,  1.     ],
       [ 0.01739,  0.86042,  1.     ],
       [-0.14444, -4.24898,  1.     ],
       [ 6.38276,  1.80984,  1.     ],
       [ 2.33197, -4.20321,  1.     ],
       [-6.6919 ,  1.80744,  1.     ],
       [-2.67265, -0.99825,  1.     ],
       [-5.1902 , -2.51251,  1.     ],
       [ 5.7403 , -0.70631,  1.     ],
       [ 2.73013, -3.5438